<a href="https://colab.research.google.com/github/Uniholder/DeepLearningSchool/blob/main/2_semester/1_NLP/%5Bhomework%5Dsimple_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
# !pip install --upgrade gensim

     |████████████████████████████████| 24.1 MB 2.7 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [15]:
import os

!mkdir data
PATH_TO_DATA = 'data'

mkdir: cannot create directory ‘data’: File exists


In [3]:
# !wget -O $PATH_TO_DATA/SO_vectors_200 https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2021-10-07 11:54:54--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘data/SO_vectors_200’

data/SO_vectors_200 100%[===================>]   1.35G  28.2MB/s    in 50s     

2021-10-07 11:55:45 (27.6 MB/s) - ‘data/SO_vectors_200’ saved [1453905423/1453905423]



In [4]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format(os.path.join(PATH_TO_DATA, 'SO_vectors_200'), binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [ ]:
# method most_simmilar
wv_embeddings.most_similar(positive='dog', topn=20)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885),
 ('canine', 0.7209057211875916),
 ('meow', 0.7185295820236206),
 ('cow', 0.7080444693565369),
 ('dog2', 0.7057910561561584),
 ('woof', 0.705061137676239),
 ('dog1', 0.7038840651512146),
 ('dog3', 0.701882004737854),
 ('penguin', 0.697029173374176),
 ('bulldog', 0.6940488815307617),
 ('mammals', 0.6931389570236206)]

In [ ]:
# косинусная схожесть между dog и cat
wv_embeddings.similarity('dog', 'cat')

0.6852341

**Ответ:**

Слово cat не входит в топ-5 близких к dog. Входит cats (множественное число).

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [5]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [6]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
    question: строка
    embeddings: наше векторное представление
    dim: размер любого вектора в нашем представлении
        
    return: векторное представление для вопроса
    """
    tokens = tokenizer.tokenize(question)
    
    v_question = np.zeros(dim)
    count = 0
    for token in tokens:
        try:
            v_question += embeddings[token]
            count += 1
        except KeyError:
            pass
        
    return v_question / count if count != 0 else v_question

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
sentence = 'I love neural networks'

v_sentence = question_to_vec(sentence, wv_embeddings, tokenizer)
v_sentence[:10]

array([-1.01422749, -1.68912624, -1.28541217, -1.37103013,  0.15916634,
        0.0670696 ,  1.11884137, -0.07051762,  0.53863693, -0.37139114])

**Ответ:**

In [ ]:
round(v_sentence[2], 2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

**Решение:**

Заменим для упрощения $rank\_q_i^{'}$ на $x_i$.

$max(\frac{1}{N}\sum_{i=1}^N \, [x_i \le 47] - \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+x_i)}\cdot[x_i \le 1]) = \\
\frac{1}{N} [max(\sum_{i=1}^N \, [x_i \le 47]) - min(\sum_{i=1}^N\frac{1}{\log_2(1+x_i)}\cdot[x_i \le 1])]$

1. $max(\sum_{i=1}^N \, [x_i \le 47])$

Максимум достигается при значениях $x_i \in [1; 47]$ для всех $i \in [1; N]$

$\sum_{i=1}^N \ 1 = N$

2. $min(\sum_{i=1}^N\frac{1}{\log_2(1+x_i)}\cdot[x_i \le 1])$

Минимум достигается при значениях $x_i \in [2, \infty)$ для всех $i \in [1; N]$

$\sum_{i=1}^N\frac{1}{\log_2(1+x_i)}\cdot 0 = 0$

Таким образом получаем:

$\frac{1}{N}(N - 0) = 1$

**Ответ:**

1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [ ]:
1 / np.log2(10)

0.3010299956639812

**Ответ:**

$\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}} \approx 0.3$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [7]:
def hits_count(dup_ranks, k):
    """
    dup_ranks: list индексов дубликатов
    result: вернуть  Hits@k
    """
    hits_value = 0
    for rank in dup_ranks:
        hits_value += (rank <= k)
    
    return hits_value / len(dup_ranks)

In [8]:
def dcg_score(dup_ranks, k):
    """
    dup_ranks: list индексов дубликатов
    result: вернуть DCG@k
    """
    dcg_value = 0
    for rank in dup_ranks:
        dcg_value += (rank <= k) / np.log2(1 + rank)
    
    return dcg_value / len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [9]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [16]:
# !gdown --id 1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_ -O $PATH_TO_DATA/stackoverflow_similar_questions.zip
# !unzip data/stackoverflow_similar_questions.zip -d data/

Downloading...
From: https://drive.google.com/uc?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_
To: /content/data/stackoverflow_similar_questions.zip
100% 131M/131M [00:01<00:00, 70.1MB/s]
Archive:  data/stackoverflow_similar_questions.zip
   creating: data/data/
  inflating: data/data/.DS_Store     
   creating: data/__MACOSX/
   creating: data/__MACOSX/data/
  inflating: data/__MACOSX/data/._.DS_Store  
  inflating: data/data/train.tsv     
  inflating: data/data/validation.tsv  


Считайте данные.

In [17]:
import os

PATH_TO_DATA = os.path.join('data', 'data')

In [18]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as f:
        for line in f.readlines():
            data.append(line.strip())
    return data

Нам понадобится только файл validation.

In [19]:
validation_data = read_corpus(os.path.join(PATH_TO_DATA, 'validation.tsv'))

Кол-во строк

In [ ]:
len(validation_data)

3760

In [ ]:
len(validation_data[0].split('\t'))

1001

In [ ]:
validation_data[0].split('\t')[1000]

'Eclipse: Most useful refactorings'

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 54516
2 52817
3 54463
4 53723
5 52296


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy
from scipy.spatial.distance import cosine

import nltk
from nltk.stem import PorterStemmer, SnowballStemmer
import spacy

In [21]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [96]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200, return_cosine=False):
    """
    для вычисления похожести используем sklearn
    question: строка
    candidates: массив строк(кандидатов) [a, b, c]
    result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    v_question = question_to_vec(question, embeddings, tokenizer, dim=dim)
    v_candidates = [question_to_vec(candidate, embeddings, tokenizer, dim=dim) for candidate in candidates]
    similarities = cosine_similarity(v_question.reshape(1, -1), v_candidates).squeeze()
    
    result = sorted(
        list(zip(range(len(candidates)), candidates, similarities)), 
        key=lambda x: x[2], 
        reverse=True
    )
    
    return result if return_cosine else [(i, c) for i, c, _ in result]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer, return_cosine=True)
    print(ranks)
    print()

[(1, 'C# create cookie from string and send it', 0.5462909752319831), (0, 'Convert Google results object (pure js) to Python object', 0.47886280839497897), (2, 'How to use jQuery AJAX for an outside domain?', 0.16188453100360892)]

[(1, 'WPF- How to update the changes in list item of a list', 0.4768912406597638), (0, 'Getting all list items of an unordered list in PHP', 0.4569547639204964), (2, 'select2 not displaying search results', 0.16329035032381012)]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

**Ответ:**

- 102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [23]:
from tqdm.notebook import tqdm

import nltk
from nltk.tokenize import (
    word_tokenize,
    TreebankWordTokenizer,
    wordpunct_tokenize,
    TweetTokenizer
)
from nltk.corpus import stopwords

In [26]:
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
ss = SnowballStemmer(language='english')
wnl = nltk.WordNetLemmatizer()

In [25]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'texcat'])

In [27]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
    используем стеммер портера
    question: строка
    embeddings: наше векторное представление
    dim: размер любого вектора в нашем представлении
        
    return: векторное представление для вопроса
    """
    tokens = tokenizer.tokenize(question.lower())
#     tokens = [wnl.lemmatize(token) for token in tokens]

    # tokens = [token.lemma_ for token in question]
    
    v_question = np.zeros(dim)
    count = 0
    tokens = [token for token in tokens if token not in stop_words]
    for token in tokens:
        try:
            v_question += embeddings[token]
            count += 1
        except KeyError:
            pass
        
    return v_question / count if count != 0 else v_question

In [56]:
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        # return re.findall('\w+', text)
        return wordpunct_tokenize(text)
tokenizer = MyTokenizer()

In [ ]:
%%time
validation_texts = []  # for spacy
for text in validation_data:
    validation_texts.append(text.split('\t'))

CPU times: user 2.26 s, sys: 258 ms, total: 2.52 s
Wall time: 2.52 s


In [ ]:
%%time
spacy_results = []  # for spacy
for texts in tqdm(validation_texts[:1000]):
    spacy_results.append(list(nlp.pipe(texts, n_process=6)))

  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 7min 37s, sys: 9min 58s, total: 17min 36s
Wall time: 18min 16s


In [57]:
%%time
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line.split('\t')
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer, return_cosine=True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

CPU times: user 1min 15s, sys: 58.9 s, total: 2min 14s
Wall time: 1min 11s


In [58]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.409 | Hits@   1: 0.409
DCG@   5: 0.499 | Hits@   5: 0.579
DCG@  10: 0.520 | Hits@  10: 0.645
DCG@ 100: 0.567 | Hits@ 100: 0.878
DCG@ 500: 0.580 | Hits@ 500: 0.973
DCG@1000: 0.583 | Hits@1000: 1.000


Результаты

Tokenizer:\w+ Prepoc:-|||Tokenizer:\w+ Prepoc:lower()||
-|-|-|-|-
DCG@   1: 0.285 | Hits@   1: 0.285||DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.342 | Hits@   5: 0.393||DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.360 | Hits@  10: 0.449||DCG@  10: 0.525 | Hits@  10: 0.651
DCG@ 100: 0.406 | Hits@ 100: 0.679||DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.431 | Hits@ 500: 0.879||DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.444 | Hits@1000: 1.000||DCG@1000: 0.586 | Hits@1000: 1.000

Приведение токенов к нижнему регистру значительно улучшает качество ранжирования. Далее будем всегда использовать данный вид препроцессинга.

Tokenizer:word_tokenize||Tokenizer:wordpunct_tokenize||Tokenizer:TreebankWordTokenizer||Tokenizer:TweetTokenizer||Tokenizer:split||
-|-|-|-|-|-|-|-|-|-
DCG@   1: 0.399 | Hits@   1: 0.399|DCG@   1: 0.410 | Hits@   1: 0.410|DCG@   1: 0.401 | Hits@   1: 0.401|DCG@   1: 0.411 | Hits@   1: 0.411|DCG@   1: 0.346 | Hits@   1: 0.346
DCG@   5: 0.487 | Hits@   5: 0.566|DCG@   5: 0.500 | Hits@   5: 0.580|DCG@   5: 0.488 | Hits@   5: 0.566|DCG@   5: 0.498 | Hits@   5: 0.577|DCG@   5: 0.432 | Hits@   5: 0.506
DCG@  10: 0.509 | Hits@  10: 0.634|DCG@  10: 0.521 | Hits@  10: 0.645|DCG@  10: 0.510 | Hits@  10: 0.635|DCG@  10: 0.517 | Hits@  10: 0.635|DCG@  10: 0.454 | Hits@  10: 0.575
DCG@ 100: 0.555 | Hits@ 100: 0.858|DCG@ 100: 0.568 | Hits@ 100: 0.875|DCG@ 100: 0.556 | Hits@ 100: 0.859|DCG@ 100: 0.563 | Hits@ 100: 0.862|DCG@ 100: 0.503 | Hits@ 100: 0.814
DCG@ 500: 0.569 | Hits@ 500: 0.969|DCG@ 500: 0.581 | Hits@ 500: 0.973|DCG@ 500: 0.570 | Hits@ 500: 0.969|DCG@ 500: 0.578 | Hits@ 500: 0.973|DCG@ 500: 0.522 | Hits@ 500: 0.960
DCG@1000: 0.573 | Hits@1000: 1.000|DCG@1000: 0.583 | Hits@1000: 1.000|DCG@1000: 0.574 | Hits@1000: 1.000|DCG@1000: 0.581 | Hits@1000: 1.000|DCG@1000: 0.526 | Hits@1000: 1.000

Лучший результат показал простой токенизатор `\w+`. Далее будем использовать его.

Prepoc:stopwords||Prepoc:PosterStemmer||Prepoc:SnowballStemmer||Prepoc:WordNetLemmatizer||Prepoc:spacy||
-|-|-|-|-|-|-|-|-|-
DCG@   1: 0.416 | Hits@   1: 0.416|DCG@   1: 0.327 | Hits@   1: 0.327|DCG@   1: 0.328 | Hits@   1: 0.328|DCG@   1: 0.407 | Hits@   1: 0.407|DCG@   1: 0.271 | Hits@   1: 0.271
DCG@   5: 0.503 | Hits@   5: 0.582|DCG@   5: 0.402 | Hits@   5: 0.471|DCG@   5: 0.404 | Hits@   5: 0.472|DCG@   5: 0.493 | Hits@   5: 0.572|DCG@   5: 0.346 | Hits@   5: 0.414
DCG@  10: 0.525 | Hits@  10: 0.651|DCG@  10: 0.423 | Hits@  10: 0.536|DCG@  10: 0.425 | Hits@  10: 0.538|DCG@  10: 0.513 | Hits@  10: 0.632|DCG@  10: 0.361 | Hits@  10: 0.460
DCG@ 100: 0.571 | Hits@ 100: 0.876|DCG@ 100: 0.476 | Hits@ 100: 0.796|DCG@ 100: 0.479 | Hits@ 100: 0.801|DCG@ 100: 0.561 | Hits@ 100: 0.867|DCG@ 100: 0.412 | Hits@ 100: 0.712
DCG@ 500: 0.584 | Hits@ 500: 0.973|DCG@ 500: 0.496 | Hits@ 500: 0.950|DCG@ 500: 0.498 | Hits@ 500: 0.951|DCG@ 500: 0.575 | Hits@ 500: 0.965|DCG@ 500: 0.436 | Hits@ 500: 0.897
DCG@1000: 0.587 | Hits@1000: 1.000|DCG@1000: 0.502 | Hits@1000: 1.000|DCG@1000: 0.504 | Hits@1000: 1.000|DCG@1000: 0.578 | Hits@1000: 1.000|DCG@1000: 0.447 | Hits@1000: 1.000

Лучший результат показал простая предобработка выравниванием регистра + удаление стоп-слов. Стемминг и лемматизация ухудшили результат.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [37]:
train_data = read_corpus(os.path.join(PATH_TO_DATA, 'train.tsv'))

In [38]:
train_data[1].split('\t')

['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?',
 'Event handling for geometries in Three.js?']

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [39]:
%%time
words = []
for row in train_data:
    line = ' '.join(row.split('\t'))
    tokens = tokenizer.tokenize(line)
    words.append(tokens)

CPU times: user 9.04 s, sys: 959 ms, total: 9.99 s
Wall time: 10 s


In [40]:
len(words)

1000000

In [89]:
%%time
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=500,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=20).wv

CPU times: user 11min 3s, sys: 1.51 s, total: 11min 4s
Wall time: 5min 43s


In [46]:
len(embeddings_trained), len(wv_embeddings), len(validation_data)

(50830, 1787145, 3760)

In [87]:
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        # return re.findall('\w+', text)
        return text.split()
tokenizer = MyTokenizer()

In [97]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
    используем стеммер портера
    question: строка
    embeddings: наше векторное представление
    dim: размер любого вектора в нашем представлении
        
    return: векторное представление для вопроса
    """
    tokens = tokenizer.tokenize(question.lower())
    # tokens = [ss.stem(token) for token in tokens]

    # tokens = [token.lemma_ for token in question]
    
    v_question = np.zeros(dim)
    count = 0
    tokens = [token for token in tokens if token not in stop_words]
    for token in tokens:
        try:
            v_question += embeddings[token]
            count += 1
        except KeyError:
            pass
        
    return v_question / count if count != 0 else v_question

In [98]:
%%time
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line.split('\t')
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer, dim=500)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

CPU times: user 1min 4s, sys: 50.1 s, total: 1min 54s
Wall time: 1min 1s


In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

Результаты

window=3||window=20||word_tokenize||WordNetLemmatizer||SnowballStemmer||
-|-|-|-|-|-|-|-|-|-
DCG@   1: 0.339 | Hits@   1: 0.339|DCG@   1: 0.342 | Hits@   1: 0.342|DCG@   1: 0.328 | Hits@   1: 0.328|DCG@   1: 0.326 | Hits@   1: 0.326|DCG@   1: 0.253 | Hits@   1: 0.253
DCG@   5: 0.416 | Hits@   5: 0.484|DCG@   5: 0.433 | Hits@   5: 0.514|DCG@   5: 0.412 | Hits@   5: 0.488|DCG@   5: 0.415 | Hits@   5: 0.497|DCG@   5: 0.331 | Hits@   5: 0.403
DCG@  10: 0.437 | Hits@  10: 0.550|DCG@  10: 0.458 | Hits@  10: 0.589|DCG@  10: 0.436 | Hits@  10: 0.563|DCG@  10: 0.441 | Hits@  10: 0.575|DCG@  10: 0.356 | Hits@  10: 0.480
DCG@ 100: 0.488 | Hits@ 100: 0.795|DCG@ 100: 0.504 | Hits@ 100: 0.818|DCG@ 100: 0.485 | Hits@ 100: 0.805|DCG@ 100: 0.484 | Hits@ 100: 0.789|DCG@ 100: 0.400 | Hits@ 100: 0.692
DCG@ 500: 0.507 | Hits@ 500: 0.946|DCG@ 500: 0.522 | Hits@ 500: 0.952|DCG@ 500: 0.503 | Hits@ 500: 0.947|DCG@ 500: 0.504 | Hits@ 500: 0.939|DCG@ 500: 0.425 | Hits@ 500: 0.887
DCG@1000: 0.513 | Hits@1000: 1.000|DCG@1000: 0.527 | Hits@1000: 1.000|DCG@1000: 0.509 | Hits@1000: 1.000|DCG@1000: 0.510 | Hits@1000: 1.000|DCG@1000: 0.437 | Hits@1000: 1.000|

Обучение эмбеддингов на обучающем датасете не улучшило результат.

Выбор размера окна: размер окна 20 незначительно улучшает результат. Возможно, это объясняется тем, что в данной задаче важен контекст всего вопроса, а вопросы состоят из +-20 токенов.

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1. Токенизация

Были рассмотрены типы:
- \w+
- word_tokenize
- wordpunct_tokenize
- TreebankWordTokenizer
- TweetTokenizer
- split

Лучший результат показала токенизация \w+ - выделение буквенных слов, чисел. Причина может быть в том, что этот метод не выделяет пунктуацию в отдельные токены.

2. Нормализация

Были опробованы следубщие методы:
- приведение к нижнему регистру
- стемминг
- лемматизация

Выравнивание регистра повысило качество ранжирования. Остальные методы - понизили. Выделение слов, имеющих заглавные буквы, в отдельные токены не имеет смысла, поэтому выравнивание помогает. Однако стемминг и лемматизация не помогли улучшить качество, а наоборот понизили его. Можно предположить что это произошло по причинам:
- качество нормализации было невысоким
- формы слов имели значение при определении схожести предложений

3. Эмбеддинги

Были опробованы 2 типа эмбеддингов:
- предобученные на StackOverFlow SO_vectors_200
- обученные на предоставленном обучающем датасете

Преобученные проявили себя лучше. Возможная причина: наши эмбеддинги обучались на гораздо больших объёмах данных

4. Качество

Возможные причины невысокого качества ранжирования:
- обучение эмбеддингов на недостаточном количестве данных
- несовпадение тематик обучающего и валидационного датасетов

5. Подход

Идея для увеличения качества:
- увеличить количество данных
- провести обучение на подходящем датасете
- применить другой подход к обучению эмбеддингов (не word2vec)